# Mô tả yêu cầu bài toán:
* Đây là bài toán phân loại các câu hỏi thiếu chân thành, những câu hỏi được đặt ra dựa trên những tiền đề sai lầm hoặc có ý định đưa ra một tuyên bố hơn là tìm kiếm những câu trả lời hữu ích trên Quora

In [ ]:
import pandas as pd
import re

# I. Data Overview

> **1. Load dữ liệu**

In [ ]:
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
train_df


**=> Nhận xét:** 

* Về yêu cầu bài toán ta thấy đây là bài toán phân lớp nhị phân, mục đích là phải detect được câu hỏi nào là toxic
* Nhìn vào question_text thì ta có thể thấy rằng đây là dữ liệu tiếng anh, mỗi mẫu là các câu hỏi, nhìn chung các câu hỏi này hỏi về vấn đề chính trị, giáo dục, giới tính,... Là chủ đề về xã hội. 
* cần phải được tiền xử lý qua các bước như đưa về dạng gốc, tokenize, loại bỏ stop words, loại bỏ dấu,...

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
labels = [ 'untoxic question', 'toxic question']
x = train_df['target'].value_counts().to_list()
y = labels
plt.xticks(fontsize =12)
plt.yticks(fontsize =12)
plt.xlabel('Polarity',fontsize =13)
plt.ylabel('Number of question',fontsize = 13)
# plt.xticks(rotation)
color_list = [ '#FF6565', '#2C99FE']
plt.bar(y,x,color = color_list,width=0.3, align='center',)
plt.title('Data Distribution')
plt.figure(figsize=(5,10))
plt.show()

**=> Nhận xét:** 
* nhìn vào biểu đồ trên ta thấy dữ liệu gặp phải vấn đề mất cân bằng. Cụ thể là nhãn có score là 1 (chính là nhãn toxic) ít hơn rất nhiều so với nhãn có score là 0 ( nhãn bình thường)

# II. Làm sạch dữ liệu:
> **1. Loại bỏ punctuation, viết thường các chữ**


In [ ]:
puncts = [
    '½', '¿', 'ï', '¸', '-', ',', '/', '"', '¨', '²', 'è', '×', '❤', '，', '↓', '▾', '↑',
    'Ã', '±', ']', '·', '_', '<', '?', '⋅', '™', '~', '→', '′', '>', '≤', '€', '¥', '¼',
    '¶', '@', '√', '®', '\\', '…', '、', '¹', '$', '•', '!', '¯', '&', '†', ')', '・', '^',
    '—', '+', '#', '（', '³', '£', '″', '−', '[', '¬', '¦', '）', '–', '”', '¢', '%', '©',
    '»', '}', '¾', '§', '=', '{', '‘', '∞', 'Ø', '°', '|', '：', '▒', 'â', 'à', ':', '(',
    ';', '`', '│', 'é', '*', '’', '.', '\'', '“',
]

def remove_punctuation(text):
    for punct in puncts:
        if punct in text:
            text = text.lower()
            text = text.replace(punct, '')
            text = text.strip()
    return text

train_df['question_text'] = train_df['question_text'].apply(remove_punctuation)
test_df['question_text'] = test_df['question_text'].apply(remove_punctuation)
train_df.question_text[0]

> **2. Loại bỏ các số**

In [ ]:
def remove_numbers(text):
    text_removed_number = ''.join([i for i in text if not i.isdigit()])
    return text_removed_number
train_df['question_text'] = train_df['question_text'].apply(remove_numbers)
test_df['question_text'] = test_df['question_text'].apply(remove_numbers)
train_df

> **2. Tokenize text**

In [ ]:

def tokenize(text):
    tokens =re.split('\W+',text)
    return tokens

train_df['question_text']= train_df['question_text'].apply(tokenize)
test_df['question_text']= test_df['question_text'].apply(tokenize)
train_df.head
    

> **3. Remove stopword:**


In [ ]:
import nltk
from nltk.corpus import stopwords

stopword =nltk.corpus.stopwords.words('english')

def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

train_df['question_text'] = train_df['question_text'].apply(remove_stopwords)
test_df['question_text'] = test_df['question_text'].apply(remove_stopwords)
train_df.head

> **4. Lemmatizing**

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

train_df['question_text'] = train_df['question_text'].apply(lambda x: lemmatizing(x))
test_df['question_text'] = test_df['question_text'].apply(lambda x: lemmatizing(x))

train_df.head()



# III. Biểu diễn dữ liệu


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder

> **1. chọn đặc trưng dùng SelectKBest với score_func là chi2**

In [ ]:
def feature_select(corpus, labels, k=1000000):
    """
    select top k features through chi-square test
    """
    bin_cv = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, stop_words=None, binary=True)
    le = LabelEncoder()
    X = bin_cv.fit_transform(corpus, np.nan)
    y = le.fit_transform(labels).reshape(-1, 1)

    skb = SelectKBest(chi2, k='all')
    skb.fit(X, y)

    feature_ids = skb.get_support(indices=True)
    feature_names = bin_cv.get_feature_names()
    result = {}
    vocab = []

    for new_fid, old_fid in enumerate(feature_ids):
        feature_name = feature_names[old_fid]
        vocab.append(feature_name)

    result['text'] = vocab
    result['_score'] = list(skb.scores_)
    result['_pvalue'] = list(skb.pvalues_)

    return result

In [ ]:
corpus = []
for question in train_df.question_text:
    corpus.append(' '.join(question))
corpus
scores = []
for score in train_df.target:
    scores.append(score)
file_out = feature_select(corpus, scores)
df = pd.DataFrame(file_out)
df = df.sort_values('_score', ascending=False)


In [ ]:
df

* **Chuẩn hóa điểm score của chi2 về trong khoảng 0 đến 1 dùng Min_max scaling**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scaling(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    scores = scaler.transform(data)
    return scores
scores = np.array(df._score)
scores = scores.reshape((len(scores),1))
df._score = scaling(scores).reshape((len(scores)))
df.to_csv('./chi2_vocabulary.csv', ',', encoding='utf-8')
df


> **2. Biểu diễn dữ liệu**

Sau khi dùng chi2 để đánh trọng số các từ, tiến hành lọc vocab theo ngưỡng điểm _score

In [ ]:
#load vocabulary
vocab_df = pd.read_csv("../input/chi2-selectkbest/chi2_vocab.csv")

X_train = []

for text in train_df.question_text:
    encoding = [v['_score'] if v['text'] in text else 0 for _,v in vocab_df.iterrows()]
    X_train.append(encoding)


In [ ]:
Y_train  = train_df.target

# IV. Huấn Luyện mô hình

> **1. Chọn model**


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, train_df.target)

> **2. Đánh giá**